In [2]:
import numpy as np, pandas as pd
periods = 1000
idx = pd.date_range("2022-01-03 09:30", periods=periods, freq="5min")
df = pd.DataFrame({
"open": np.linspace(100,110,periods),
"high": np.linspace(101,111,periods),
"low": np.linspace( 99,109,periods),
"close": np.linspace(100,110,periods),
"volume": np.random.randint(1000,5000,periods),
}, index=idx)
# normalize column names and ensure ascending datetime index
df = df.rename(columns=str.lower).sort_index()
df

,open,high,low,close,volume
2022-01-03 09:30:00,100.00000,101.00000,99.00000,100.00000,1295
2022-01-03 09:35:00,100.01001,101.01001,99.01001,100.01001,3427
2022-01-03 09:40:00,100.02002,101.02002,99.02002,100.02002,2918
2022-01-03 09:45:00,100.03003,101.03003,99.03003,100.03003,3160
2022-01-03 09:50:00,100.04004,101.04004,99.04004,100.04004,2332
...,...,...,...,...,...
2022-01-06 20:25:00,109.95996,110.95996,108.95996,109.95996,4469
2022-01-06 20:30:00,109.96997,110.96997,108.96997,109.96997,2188
2022-01-06 20:35:00,109.97998,110.97998,108.97998,109.97998,2311
2022-01-06 20:40:00,109.98999,110.98999,108.98999,109.98999,4097


In [3]:
FEATURE_CODE = "<feature_code>" # e.g., "trend_sma_20"
import numpy as np
import pandas as pd
def compute_feature(df: pd.DataFrame) -> pd.Series:
    """
    <Human-readable name> (e.g., "Simple Moving Average (20)")
    Description:
    What this measures and why it might help.
    Formula / method (brief, cite if needed):
    Short formula or algorithm outline.
    Input:
    df: DataFrame with DatetimeIndex (ascending), columns:
    open, high, low, close, volume (case-insensitive)
    Output:
    pd.Series (float preferred), same index as df.index, length == len(df),
    name == FEATURE_CODE. Initial NaNs from rolling windows are OK.
    Constraints:
    - No look-ahead (use only current and past data at each row).
    - Vectorized (avoid Python loops when possible).
    - Use only numpy and pandas.
    """
    g = df.copy()
    g.columns = [str(c).lower() for c in g.columns] # normalize
    # --- your logic here; example for SMA(20) ---
    s = g["close"].rolling(20, min_periods=20).mean()
    s = s.astype(float)
    s.name = FEATURE_CODE
    return s